# Booking to EZskin

EZskin is a very popular clinic, every reservation starts at 12:00 am in the midnight, and it usually ends up in fewer seconds. To get the reservation, This program using Selenium script in python to auto book it.

#### Author : Wake Liu

<table>
<td>
<img src="pic1.png" style="width:300;height:450px;">

</td>

</table>

- clock_setting: 若為False，直接執行該程式
- times:重刷次數
- interval等待時間

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
import time

clock_setting = True
hour, minute, sec = 23, 59, 59
times = 9
interval = 0
username = "A272152421" #輸入自己的身分證字號
password = "password"

#### 由於每個ID都是獨立的，我們可以在頁面中找到自己要的ID資訊。
#### 以EZskin為例，在登入時所需要的資料為ID, Password, Login, 從網站原始碼截取的資訊如下

In [2]:
def login_proc(username, password):
    login_url = "https://www.ezskin.com.tw/member/member_login.aspx"
    sel = webdriver.Chrome()
    sel.get("https://www.ezskin.com.tw/member/member_login.aspx")
    # sign in the username
    try:
        user_input = sel.find_element_by_id("ctl00_ContentPlaceHolder1_txtUserID")
        user_input.clear()
        user_input.send_keys(username)
        print ('user-id write success!')
    except:
        print ('user-id write error!')
    # sign in the pasword
    try:
        pwd_input = sel.find_element_by_id("ctl00_ContentPlaceHolder1_txtPassword")
        pwd_input.clear()
        pwd_input.send_keys(password)
        print ('pw write success!')
    except:
        print ('pw write error!')
    # Check for Login success
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnLogin").click()
    except:
        print ('Login Failed')

    while True:
        curpage_url = sel.current_url
        if curpage_url != login_url:
            if curpage_url[:-1] != login_url:  # choose wrong verify_pic
                print ('Login finished!')
                break
        else:
            time.sleep(5)
            print ('Login Failed!')
    return sel

In [3]:
sel = login_proc(username, password)

user-id write success!
pw write success!
Login finished!


#### Since 門診掛號沒有id,這邊用Xpath的方式將該element 找到

In [4]:
try:
    sel.find_element_by_xpath('//*[@id="body"]/div[4]/div[2]/div[1]/div/div[2]/table/tbody/tr/td/table/tbody/tr[4]/td/div/input').click()
    print ("準備門診掛號...")
except BaseException:
    print ("無法門診掛號...")

準備門診掛號...


設定起始日，當日子有更動時代表網頁已經更新，就可以進行點選。

In [5]:
start_date = sel.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_dbOutpatient"]/tbody/tr[9]/td[1]').text

#### 仔細觀察，該checkbox有固定的格式

#### 如果有出現該id就直接click然後break 離開迴圈

In [6]:
def select_proc():
    try:
        sel.refresh()
    except:
        print ("Cannot Access this website")
    current_date = sel.find_element_by_xpath('//*[@id="ctl00_ContentPlaceHolder1_dbOutpatient"]/tbody/tr[9]/td[1]').text
    if start_date != current_date:
        print (current_date,"booking is opening...")
        for i in range (1,4):
            checkbox = 'ctl00_ContentPlaceHolder1_dbOutpatient_ctl09_chkSelect'+str(i)
            try:   
                sel.find_element_by_id(checkbox).click()
                return 1
            except BaseException:
                continue
            #print(checkbox + " element not exist")
    return 0

#### 這個步驟是等待，我們只有在23時59分59秒才需要開啟該程式，其他時間都可以休息

In [7]:
while(clock_setting):
    localtime = time.localtime(time.time())
    if localtime.tm_hour == hour and localtime.tm_min == minute and localtime.tm_sec == sec:
        while(1):
            localtime = time.localtime(time.time())
            if localtime.tm_sec == (sec+1)%60 :
                sleep(interval)
                break
        break
    print (localtime.tm_hour,localtime.tm_min,localtime.tm_sec)
    if localtime.tm_hour != hour:
        sleep(3600)
    elif localtime.tm_min != minute:
        sleep(60)
    else:
        sleep(1)



23 48 32
23 49 32
23 50 32
23 51 32
23 52 32
23 53 32
23 54 32
23 55 32
23 56 32
23 57 32
23 58 32
23 59 32
23 59 33
23 59 34
23 59 35
23 59 36
23 59 37
23 59 38
23 59 39
23 59 40
23 59 41
23 59 42
23 59 43
23 59 44
23 59 45
23 59 46
23 59 47
23 59 48
23 59 49
23 59 50
23 59 51
23 59 52
23 59 53
23 59 55
23 59 56
23 59 57
23 59 58


時間到時，該程式一共會刷新9次（因為一分鐘內刷新第十次會被網站封鎖），每次執行select_proc()的動作。

In [8]:
current_time = time.time()
for i in range(times):
    flag = select_proc()
    time_cost = time.time()-current_time
    current_time = time.time()
    if flag == True:
        print ("Successfully Selected, the time is i = ",i, time.localtime(current_time),"cost = ",time_cost)
        break
    else :
        print ("Keep Waiting, the time is i = ",i, time.localtime(current_time),"cost = ",time_cost)
        

3/31(六) booking is opening...
Successfully Selected, the time is i =  0 time.struct_time(tm_year=2018, tm_mon=3, tm_mday=24, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=5, tm_yday=83, tm_isdst=0) cost =  0.7809939384460449


下一步跟確認掛號都算是簡單的，如果有成功掛號(即flag為true)，只要抓到id繼續click就可以了

##### 先點擊下一步
<input type="submit" name="ctl00$ContentPlaceHolder1$btnNext" value="下一步" onclick="return checkModify();" id="ctl00_ContentPlaceHolder1_btnNext" class="btn01">

##### 確認掛號語法

這邊還是要碰一點運氣，有機會鍛羽而歸

<input type="submit" name="ctl00$ContentPlaceHolder1$btnSave" value="確定掛號" id="ctl00_ContentPlaceHolder1_btnSave" class="btn01">

In [9]:
if flag == True:
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnNext").click()
        print ("Successfully Next Step!")
    except BaseException:
        print ("Cannot Press Next step!")
    try:
        sel.find_element_by_id("ctl00_ContentPlaceHolder1_btnSave").click()
        print ("Successfully Saved!")
    except BaseException:
        print ("Cannot save, 應用程式發生錯誤")

    try:
        msg = sel.switch_to_alert().text
        if '您已經在此門診掛號' in msg:
            print (msg,"Congratulation!!! Your reservation is done!")
        else:
            print (msg,"Sorry >_< all the doctors are reserved!")
        sel.switch_to_alert().accept()
    except BaseException:
        None
else:
    print ("Sorry >_< all the doctors are reserved!")

Successfully Next Step!
Successfully Saved!


In [10]:
localtime = time.localtime(time.time())
print ("Current Time is"+str(localtime))

Current Time istime.struct_time(tm_year=2018, tm_mon=3, tm_mday=24, tm_hour=0, tm_min=0, tm_sec=14, tm_wday=5, tm_yday=83, tm_isdst=0)
